In [1]:
import pandas
import email
from sklearn.feature_extraction.text import CountVectorizer
import html2text

dataset = pandas.read_msgpack('./data/development.msg', encoding='latin-1')
dataset['email'] = dataset['email'].apply(email.message_from_string)


In [3]:
corpus = []
for (index, row) in dataset.iterrows():
    if row['class'] == 'spam':
        for part in row['email'].walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload()
            elif part.get_content_type() == 'text/html':
                body = html2text.html2text(part.get_payload())
        
        corpus.append(body)

In [4]:
import io

vectorizer = CountVectorizer(min_df=1)
words_model = vectorizer.fit_transform(corpus)

MemoryError: 

In [12]:
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components=200)
pca_model = pca.fit_transform(words_model)

In [11]:
pca.transform(vectorizer.transform(corpus[0]))

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

36091
200


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline



class FeatureTransformer(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def generate(x):
        return x
    
    def transform(self, data):
        features = []
        
        for entry in data:
            features.append(self.generate(data))
        
        return features
    
# Converts textual emails to python email objects
class EmailGenerator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, data):
        import email
        data['email'] = data['email'].apply(email.message_from_string)
        return data

class ContentTypeGenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, data):
        check = ['x-world', 'application', 'text', 'text/plain', 'text/html', 'video', 'audio', 'image', 'drawing', 'model', 'multipart', 'x-conference', 'i-world', 'music', 'message', 'x-music', 'www', 'chemical', 'paleovu', 'windows', 'xgl']
        features = []
        
        for entry in data:
            email = entry['email']
            output = collections.defaultdict(bool)

            for part in email.walk():
                ct = part.get_content_type()

                for kind in check:
                    output['has_' + kind] |= ct.startswith(kind)
            
            features.append(output)
        
        return features
    
    
class EmailCountsGenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, data):
        features = []
        
        for entry in data:
            email = str(entry['email'])
            
            output = {
                'length': len(email),
                'spaces': email.count(' '),
                'newlines': email.count('\n')
            }
            
            features.append(output)
        
        return features

class ContentTypeGenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, data):
        features = []
        
        for entry in data:
            email = entry['email']
            output = {}
            
            features.append(output)
        
        return features
    
class FeatureGeneratorBase(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, data):
        features = np.recarray(shape=(len(data),),
            dtype=[('subject', object), ('body', object)])
        
        return features

    
pipeline = Pipeline([
        ('transform_email', EmailGenerator()),
        ('features', FeatureUnion=(
                transformer_list=[
                    ('content_type_features', ContentTypeGenerator())
                ]
            ))
    ])